# Hybrid Quantum–Classical AI — Full end-to-end workflow

High-level idea: Hybrid systems combine classical computing (CPU/GPU) and quantum processing units (QPU). Typical pattern: do heavy preprocessing, feature extraction and data engineering classically; send the smaller, computationally intensive or expressivity-critical sub-problem (a variational quantum circuit, kernel evaluation, or quantum subroutine) to the QPU; merge quantum outputs with classical postprocessing and use a classical optimizer. This is practical today and the dominant approach for near-term quantum advantage.

# Problem statement

Build a hybrid quantum–classical classifier to predict breast cancer diagnosis (benign vs malignant). Preprocess and extract features classically; encode reduced features into qubits; use a variational quantum circuit as a classifier (quantum feature map + parametrized ansatz). Use classical optimizer to train parameters.

# Import basic libraries

In [ ]:
# Core Python libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn for dataset, preprocessing, model selection, classical baselines
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Classical model for baseline
from sklearn.ensemble import RandomForestClassifier

# PennyLane for quantum circuits (hybrid)
import pennylane as qml
from pennylane import numpy as pnp

# Load data

In [5]:
data_obj = load_breast_cancer(as_frame=True)
data = data_obj.frame.copy()
data['target'] = data_obj.target
data.head()

NameError: name 'load_breast_cancer' is not defined

# Domain analysis

Breast cancer dataset: 30 numerical features describing cell nuclei (mean, se, worst) and binary target (0 = malignant, 1 = benign depending on loader mapping). Commonly used to teach classification. High-dimensional (30 features) — good candidate to apply classical dimensionality reduction before quantum encoding because near-term QPUs support few qubits.

# Basic checks

In [8]:
print("Shape:", data.shape)
print(data.info())
print(data.describe().T)
print("Missing values:\n", data.isnull().sum())

NameError: name 'data' is not defined

# Exploratory data analysis EDA

In [ ]:
# Class balance
data['target'].value_counts(normalize=True)

# Correlation heatmap (show a subset because full 30x30 is large)
plt.figure(figsize=(10,8))
sns.heatmap(data.iloc[:, :12].corr(), annot=False, cmap='coolwarm')
plt.title("Feature correlation (subset)")
plt.show()

# Simple pair plot for a few top features
cols = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'target']
sns.pairplot(data[cols], hue='target', corner=True)
plt.show()

 Insight: Many features are correlated and separable; reducing dimensionality is beneficial before encoding to qubits.

# Feature engineering

Use classical feature selection / PCA to reduce to a small number of features (e.g., 4 features → 4 qubits).

You can also compute domain features (ratios, aggregated metrics), but here use PCA for simplicity.

In [ ]:
X = data.drop('target', axis=1).values
y = data['target'].values  # 0/1 labels

# Classical scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use PCA to reduce to 4 components (adjust to number of qubits you want)
from sklearn.decomposition import PCA
n_components = 4
pca = PCA(n_components=n_components, random_state=42)
X_reduced = pca.fit_transform(X_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_.sum())

# Data preprocessing

In [ ]:
# Train-test split and optional MinMax scaling to [0, π] for angle embedding.
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.25, random_state=42, stratify=y
)

# Map to [0, pi] for angle encoding
minmax = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = minmax.fit_transform(X_train)
X_test_scaled = minmax.transform(X_test)

# Hybrid design decision — what runs classically vs quantumly

Classical (CPU/GPU):

Feature scaling, PCA, heavy feature extraction, dataset management, logging, hyperparameter search.

Loss computation, parameter update logic (optimizer) runs classically (unless using QML optimizers hosted on QPU providers).

Quantum (QPU or simulator):

Parameterized quantum circuit (feature map + variational ansatz) that outputs an expectation value / probability used by the classical optimizer.

Hybrid loop:

For each training batch: classical prepares batch → passes features to quantum circuit (encode & run) → quantum returns expectation values → classical computes loss and requests parameter updates → repeat.

We implement this with PennyLane default.qubit simulator for demonstration. Switching to a real QPU requires changing qml.device to the provider device (e.g., qml.device('qiskit.ibmq', ...) or Pennylane plugin for Xanadu/IonQ), credentials and smaller circuit depth.

# Quantum model building (feature map + variational ansatz)

We use AngleEmbedding (map feature values to rotation angles) and a simple layered variational ansatz with entanglement. Output is expectation of Z on one or more wires; map to probability.

In [ ]:
n_qubits = n_components  # 4 in this example
dev = qml.device("default.qubit", wires=n_qubits)  # simulator; replace for real QPU

# Define ansatz: angle embedding followed by parameterized rotations and entanglers
def quantum_circuit(params, x):
    # x length must equal n_qubits
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    # ansatz: one layer of single-qubit rotations followed by entangling CNOT chain; can use multiple layers
    n_layers = params.shape[0]
    for layer in range(n_layers):
        for w in range(n_qubits):
            qml.RY(params[layer, w], wires=w)
        # entangling ring
        for w in range(n_qubits - 1):
            qml.CNOT(wires=[w, w+1])
        qml.CNOT(wires=[n_qubits-1, 0])

@qml.qnode(dev, interface='autograd')
def qnode(params, x):
    quantum_circuit(params, x)
    # return expectation values; we return the mean Z across a wire (or vector)
    return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

We choose to reduce the output vector to a single score (linear combination) which is fed to a classical sigmoid to produce probability.

In [ ]:
# Initialize parameters: shape (n_layers, n_qubits)
n_layers = 2
params_init = pnp.random.normal(scale=0.1, size=(n_layers, n_qubits), requires_grad=True)

# Classical weights to map quantum outputs to single logit
classical_w = pnp.random.normal(scale=0.1, size=(n_qubits,), requires_grad=True)
classical_b = pnp.array(0.0, requires_grad=True)

def quantum_forward(params, x, w, b):
    expvals = qnode(params, x)               # returns length-n_qubits list/array
    expvals = pnp.array(expvals)
    logit = pnp.dot(w, expvals) + b
    prob = 1 / (1 + pnp.exp(-logit))
    return prob

#### Training — hybrid loop (classical optimizer updates parameters using quantum evaluations)

We will use a classical optimizer (Adam) to update both quantum parameters and the small classical layer weights. In each training iteration the quantum circuit is executed for each sample (or a small batch). For real QPU usage you must batch measurements, group commuting observables, or use gradient-free optimizers to reduce QPU calls.

In [ ]:
# loss: binary cross-entropy
def loss_fn(params, w, b, X_batch, y_batch):
    preds = [quantum_forward(params, x, w, b) for x in X_batch]
    preds = pnp.array(preds)
    y_arr = pnp.array(y_batch, dtype=float)
    # avoid log 0
    eps = 1e-9
    loss = -pnp.mean(y_arr * pnp.log(preds + eps) + (1 - y_arr) * pnp.log(1 - preds + eps))
    return loss

# optimizer: classical Adam (PennyLane provides gradient support)
opt = qml.AdamOptimizer(stepsize=0.05)

# Pack parameters together for convenience or update separately
params = params_init.copy()
w = classical_w.copy()
b = classical_b.copy()

# training hyperparameters
epochs = 20
batch_size = 16
num_samples = X_train_scaled.shape[0]

for epoch in range(1, epochs + 1):
    # shuffle
    perm = np.random.permutation(num_samples)
    X_shuf = X_train_scaled[perm]
    y_shuf = y_train[perm]
    for i in range(0, num_samples, batch_size):
        X_batch = X_shuf[i:i+batch_size]
        y_batch = y_shuf[i:i+batch_size]
        
        # classical optimizer step on combined variables using closure
        def closure(packed):
            p, ww, bb = packed
            return loss_fn(p, ww, bb, X_batch, y_batch)
        
        # Update weights: update quantum params and classical weights sequentially (common hybrid pattern)
        params = opt.step(lambda p: loss_fn(p, w, b, X_batch, y_batch), params)
        w = opt.step(lambda ww: loss_fn(params, ww, b, X_batch, y_batch), w)
        b = opt.step(lambda bb: loss_fn(params, w, bb, X_batch, y_batch), b)
    
    # compute epoch loss
    train_loss = loss_fn(params, w, b, X_train_scaled, y_train)
    if epoch % 2 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} — Train loss: {train_loss:.4f}")

#### Notes about QPU usage:

On a real QPU, each qnode call triggers quantum executions and measurements. To reduce QPU time:

Use batched circuits / vectorized execution where supported.

Use fewer parameters / shallow circuits.

Use gradient-free optimizers or parameter-shift rules with measurement grouping.

Use hardware-compatible gates and transpilation.

## Predictions

In [ ]:
y_prob_test = [quantum_forward(params, x, w, b) for x in X_test_scaled]
y_pred_test = np.array([1 if p >= 0.5 else 0 for p in y_prob_test])

# Evaluation

In [ ]:
print("Hybrid Q-C Accuracy:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
cm = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#### Compare to classical baseline:

In [ ]:
# Classical baseline: RandomForest on the same reduced features
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_cl = clf.predict(X_test)
print("Classical RF accuracy:", accuracy_score(y_test, y_cl))
print(classification_report(y_test, y_cl))

#  Conclusion

The hybrid pipeline demonstrates the standard hybrid pattern: heavy preprocessing and dimensionality reduction done classically; a small quantum circuit provides an expressive classifier mapped to a classical output layer.

For near-term devices, keeping the quantum circuit shallow and number of qubits small is essential. Choosing which subproblem to offload is the key design decision.

On small datasets, classical baselines often match or exceed hybrid Q-C accuracy. The goal of hybrid approaches today is to explore expressivity and prepare for future hardware where certain tasks may benefit from quantum resources.

In [25]:
print("Mohit Janbandhu")

Mohit Janbandhu


# Full compact script (hybrid_qc_breastcancer.py)

In [ ]:
# hybrid_qc_breastcancer.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import pennylane as qml
from pennylane import numpy as pnp

# Load and preprocess
data_obj = load_breast_cancer(as_frame=True)
data = data_obj.frame.copy()
data['target'] = data_obj.target
X = data.drop('target', axis=1).values
y = data['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_components = 4
pca = PCA(n_components=n_components, random_state=42)
X_reduced = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.25, random_state=42, stratify=y)

minmax = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = minmax.fit_transform(X_train)
X_test_scaled = minmax.transform(X_test)

# Quantum setup
n_qubits = n_components
dev = qml.device("default.qubit", wires=n_qubits)
n_layers = 2

def quantum_circuit(params, x):
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')
    for layer in range(params.shape[0]):
        for w in range(n_qubits):
            qml.RY(params[layer, w], wires=w)
        for w in range(n_qubits - 1):
            qml.CNOT(wires=[w, w+1])
        qml.CNOT(wires=[n_qubits-1, 0])

@qml.qnode(dev, interface='autograd')
def qnode(params, x):
    quantum_circuit(params, x)
    return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

# Initialize parameters
params = pnp.random.normal(scale=0.1, size=(n_layers, n_qubits), requires_grad=True)
w = pnp.random.normal(scale=0.1, size=(n_qubits,), requires_grad=True)
b = pnp.array(0.0, requires_grad=True)

def quantum_forward(params, x, w, b):
    expvals = pnp.array(qnode(params, x))
    logit = pnp.dot(w, expvals) + b
    prob = 1 / (1 + pnp.exp(-logit))
    return prob

def loss_fn(params, w, b, X_batch, y_batch):
    preds = [quantum_forward(params, x, w, b) for x in X_batch]
    preds = pnp.array(preds)
    y_arr = pnp.array(y_batch, dtype=float)
    eps = 1e-9
    loss = -pnp.mean(y_arr * pnp.log(preds + eps) + (1 - y_arr) * pnp.log(1 - preds + eps))
    return loss

opt = qml.AdamOptimizer(stepsize=0.05)
epochs = 20
batch_size = 16
num_samples = X_train_scaled.shape[0]

for epoch in range(1, epochs + 1):
    perm = np.random.permutation(num_samples)
    X_shuf = X_train_scaled[perm]
    y_shuf = y_train[perm]
    for i in range(0, num_samples, batch_size):
        X_batch = X_shuf[i:i+batch_size]
        y_batch = y_shuf[i:i+batch_size]
        params = opt.step(lambda p: loss_fn(p, w, b, X_batch, y_batch), params)
        w = opt.step(lambda ww: loss_fn(params, ww, b, X_batch, y_batch), w)
        b = opt.step(lambda bb: loss_fn(params, w, bb, X_batch, y_batch), b)
    train_loss = loss_fn(params, w, b, X_train_scaled, y_train)
    if epoch % 2 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} — Train loss: {train_loss:.4f}")

# Evaluate hybrid model
y_prob_test = [quantum_forward(params, x, w, b) for x in X_test_scaled]
y_pred_test = np.array([1 if p >= 0.5 else 0 for p in y_prob_test])
print("Hybrid Q-C Accuracy:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

# Classical baseline
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_cl = clf.predict(X_test)
print("Classical RF accuracy:", accuracy_score(y_test, y_cl))
print(classification_report(y_test, y_cl))
